In [1]:
import pandas as pd
import os
import yaml
import collections
import numpy as np
import math

# Load data

In [2]:
SACC_FOLDER = os.path.join('..','Dane trackingowe', 'sacc')
BEH_FOLDER = os.path.join('..','results', 'beh')
FIX_FOLDER = os.path.join('..', 'Dane trackingowe', 'fix')
RAW_FOLDER = os.path.join('..', 'Dane trackingowe', 'raw')
YAML_FOLDER = os.path.join('..', 'results', 'yaml')

In [3]:
sacc_files = os.listdir(SACC_FOLDER)
beh_files = os.listdir(BEH_FOLDER)
fix_files = os.listdir(FIX_FOLDER)
raw_files = os.listdir(RAW_FOLDER)
yaml_files = os.listdir(YAML_FOLDER)

In [1]:
part_id = sacc_files[34].split('_')[0]
sacc_data = pd.read_csv(os.path.join(SACC_FOLDER, part_id + '_sacc.csv')).drop('Unnamed: 0', 1)
beh_data = pd.read_csv(os.path.join(BEH_FOLDER, part_id + '_beh.csv'))
fix_data = pd.read_csv(os.path.join(FIX_FOLDER, part_id + '_fix.csv')).drop('Unnamed: 0', 1)
raw_data = pd.read_csv(os.path.join(RAW_FOLDER, part_id + '_raw.csv')).drop('Unnamed: 0', 1)
yaml_data = yaml.load(open(os.path.join(YAML_FOLDER, part_id + '.yaml'), 'r'))
print part_id

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(part_id)? (<ipython-input-1-b1137ce6c923>, line 7)

# Remove trainig trials

In [5]:
# training_trials_no = sum(beh_data.exp == 'training')
# beh_data = beh_data.ix[training_trials_no:]
# fix_data = fix_data[fix_data.block > training_trials_no]
# raw_data = raw_data[raw_data.block > training_trials_no]

In [6]:
beh_data.head()

,session_id,start_time,end_time,choosed_option,ans_accept,rt,corr,time,rel,feedb,wait,exp,type,answers
0,1,912.015100,1062.003110,-1,False,-1.000000,False,150,3,1,5,training,trial,"[1, 2, 3, 4, 5, 6]"
1,1,1062.836508,1191.173823,D1,True,128.335384,True,150,3,1,5,training,trial,"[1, 2, 3, 4, 5, 6]"
2,1,1191.940560,1274.593100,D2,True,82.651217,False,150,4,1,5,training,trial,"[1, 2, 3, 4, 5, 6]"
3,1,1297.660501,1346.678674,D1,True,49.006387,True,120,3,0,5,experiment,trial,"[1, 4, 4, 5, 5, 6]"
4,1,1347.462066,1391.013404,D1,True,43.539309,True,120,4,0,5,experiment,trial,"[1, 3, 3, 4, 4, 6]"


In [7]:
fix_data.head()

,stime,etime,dur,axp,ayp,aps,eye,block
0,1447070,1447093,24,980.6,133.9,801,R,1
1,1447172,1447294,123,1082.1,371.4,825,R,1
2,1447379,1447510,132,82.8,229.8,867,R,1
3,1447562,1447878,317,35.1,20.1,871,R,1
4,1447940,1448077,138,568.7,264.4,910,R,1


In [8]:
raw_data.head()

,time,xp,yp,ps,cr.info,block
0,1447063.0,979.2,131.5,801.0,...,1
1,1447064.0,979.2,131.5,801.0,...,1
2,1447065.0,979.4,131.5,801.0,...,1
3,1447066.0,979.9,131.7,801.0,...,1
4,1447067.0,980.9,132.2,801.0,...,1


# ROI's definitions

In [9]:
ROIS = {
    'P1' : [(-  15,  280), ( 235,   30)],
    'P2' : [(-  15,  680), ( 235,  430)],
    'P3' : [(-  15, 1080), ( 235,  830)],
    'A'  : [(+ 465,  520), ( 715,  270)],
    'B'  : [(+ 955,  520), (1205,  270)],
    'C'  : [(+1445,  520), (1695,  270)],
    'D'  : [(+ 465,  870), ( 715,  630)],
    'E'  : [(+ 955,  870), (1205,  630)],
    'F'  : [(+1445,  870), (1695,  630)]
}    

def in_roi(df, roi):
    """Assume that RECTANGLED roi is defined as list [a, b] where:
    a - coordinate of upper left corner (x1, y1)
    b - coordinate of lower right corner (x2, y2)
    x, y - point in space
    """
    x, y = df.columns
    [(x1, y1), (x2, y2)] = roi
    return (df[x] > x1) & (df[x] < x2) & (df[y] < y1) & (df[y] > y2)


# toggle rate (TR) 
based on the number of saccades running either way between the matrix area and the response area (number of toggles) divided by total response time in seconds (i.e., toggles per second). 

In [10]:
sacc_start_in_question_area = pd.concat([in_roi(sacc_data[['sxp', 'syp']], ROIS['P1']),
                                         in_roi(sacc_data[['sxp', 'syp']], ROIS['P2']),
                                         in_roi(sacc_data[['sxp', 'syp']], ROIS['P3'])], axis=1).any(axis = 1)

sacc_ends_in_matrix_area = pd.concat([in_roi(sacc_data[['exp', 'eyp']], ROIS['A']),
                                      in_roi(sacc_data[['exp', 'eyp']], ROIS['B']),
                                      in_roi(sacc_data[['exp', 'eyp']], ROIS['C']),
                                      in_roi(sacc_data[['exp', 'eyp']], ROIS['D']),
                                      in_roi(sacc_data[['exp', 'eyp']], ROIS['E']),
                                      in_roi(sacc_data[['exp', 'eyp']], ROIS['F'])], axis=1).any(axis = 1)

sacc_starts_in_question_area_and_ends_in_matrix_area = pd.concat([sacc_start_in_question_area, sacc_ends_in_matrix_area], axis = 1).all(axis = 1)

In [11]:
sacc_start_in_matrix_area = pd.concat([in_roi(sacc_data[['sxp', 'syp']], ROIS['A']),
                                       in_roi(sacc_data[['sxp', 'syp']], ROIS['B']),
                                       in_roi(sacc_data[['sxp', 'syp']], ROIS['C']),
                                       in_roi(sacc_data[['sxp', 'syp']], ROIS['D']),
                                       in_roi(sacc_data[['sxp', 'syp']], ROIS['E']),
                                       in_roi(sacc_data[['sxp', 'syp']], ROIS['F'])], axis=1).any(axis = 1)

sacc_ends_in_question_area = pd.concat([in_roi(sacc_data[['exp', 'eyp']], ROIS['P1']),
                                        in_roi(sacc_data[['exp', 'eyp']], ROIS['P1']),
                                        in_roi(sacc_data[['exp', 'eyp']], ROIS['P1'])], axis=1).any(axis = 1)

sacc_starts_in_matrix_area_and_ends_in_question_area = pd.concat([sacc_start_in_matrix_area, sacc_ends_in_question_area], axis = 1).all(axis = 1)


In [12]:
toggled_sacc = pd.concat([sacc_starts_in_matrix_area_and_ends_in_question_area, sacc_starts_in_question_area_and_ends_in_matrix_area], axis = 1).any(axis = 1)
toggled_sacc = sacc_data[toggled_sacc]
toggled_sacc.head()

,stime,etime,dur,sxp,syp,exp,eyp,ampl,pv,eye,block
1,1447295,1447378,84,1085.3,370.3,75.2,223.4,31.08,751,R,1
17,1451645,1451701,57,28.5,74.9,578.5,316.1,17.75,909,R,1
18,1452124,1452185,62,592.0,330.3,58.3,51.4,18.04,604,R,1
84,1472475,1472534,60,72.6,487.9,566.8,322.1,15.74,695,R,1
89,1473680,1473735,56,125.2,181.9,572.6,292.9,13.91,738,R,1


In [13]:
ts = toggled_sacc.groupby('block').count()['stime']
missing_blocks = set(range(1, 46)) - set(ts.index)
for i in list(missing_blocks):
    ts.ix[i] = 0 
ts.sort_index(inplace=True)
ts = ts.reset_index()
beh_data['no_toggles'] = ts['stime']
beh_data.tail()

,session_id,start_time,end_time,choosed_option,ans_accept,rt,corr,time,rel,feedb,wait,exp,type,answers,no_toggles
40,1,3500.046150,3592.215688,D1,True,92.160427,True,120,3,0,5,experiment,trial,"[1, 2, 2, 3, 3, 6]",18
41,1,3592.982379,3658.634434,D1,True,65.647010,True,120,4,0,5,experiment,trial,"[1, 2, 2, 3, 3, 6]",16
42,1,3659.384465,3691.702099,D1,True,32.303809,True,120,3,0,5,experiment,trial,"[1, 4, 4, 5, 5, 6]",10
43,1,3692.468843,3712.252780,D1,True,19.777563,True,120,3,0,5,experiment,trial,"[1, 4, 4, 5, 5, 6]",4
44,1,3713.036147,3744.937131,D1,True,31.892771,True,120,4,0,5,experiment,trial,"[1, 3, 3, 4, 4, 6]",5


In [15]:
toggles = beh_data.groupby('answers').sum()
w = (toggles['no_toggles'] / toggles['rt'])
w

answers
[1, 2, 2, 3, 3, 6]    0.184221
[1, 2, 3, 4, 5, 6]    0.447648
[1, 3, 3, 4, 4, 6]    0.171075
[1, 4, 4, 5, 5, 6]    0.290758
dtype: float64

In [16]:
toggles['no_toggles']

answers
[1, 2, 2, 3, 3, 6]    183
[1, 2, 3, 4, 5, 6]     94
[1, 3, 3, 4, 4, 6]    132
[1, 4, 4, 5, 5, 6]    130
Name: no_toggles, dtype: int64

# relative time on matrix (RTM) 
summed duration of all fixations within the matrix area (time on matrix) divided by total response time. 

In [17]:
fix_in_matrix_area = pd.concat([in_roi(fix_data[['axp', 'ayp']], ROIS['P1']),
                                in_roi(fix_data[['axp', 'ayp']], ROIS['P2']),
                                in_roi(fix_data[['axp', 'ayp']], ROIS['P2'])], axis = 1).any(axis = 1)

In [18]:
fix_in_matrix_area_on_block_dur = fix_data[fix_in_matrix_area].groupby('block').sum()['dur']

missing_blocks = set(range(1, 46)) - set(fix_in_matrix_area_on_block_dur.index)
for i in list(missing_blocks):
     fix_in_matrix_area_on_block_dur.ix[i] = 0 
fix_in_matrix_area_on_block_dur.sort_index(inplace=True)
fix_in_matrix_area_on_block_dur = fix_in_matrix_area_on_block_dur.reset_index()

beh_data['fix_in_matrix_area_dur'] = fix_in_matrix_area_on_block_dur['dur']

gb = beh_data.groupby('answers').sum()
w = (gb['fix_in_matrix_area_dur'] / (gb['rt'] * 1000.0))
w

answers
[1, 2, 2, 3, 3, 6]    0.326402
[1, 2, 3, 4, 5, 6]    0.501946
[1, 3, 3, 4, 4, 6]    0.260066
[1, 4, 4, 5, 5, 6]    0.422429
dtype: float64

# 3. relative first response fixation (RFRF) 
timestamp of the first fixation within the response area (time before first response fixation) divided by total response time.

In [19]:
fix_data.head()

,stime,etime,dur,axp,ayp,aps,eye,block
0,1447070,1447093,24,980.6,133.9,801,R,1
1,1447172,1447294,123,1082.1,371.4,825,R,1
2,1447379,1447510,132,82.8,229.8,867,R,1
3,1447562,1447878,317,35.1,20.1,871,R,1
4,1447940,1448077,138,568.7,264.4,910,R,1


In [20]:
raw_data.head()

,time,xp,yp,ps,cr.info,block
0,1447063.0,979.2,131.5,801.0,...,1
1,1447064.0,979.2,131.5,801.0,...,1
2,1447065.0,979.4,131.5,801.0,...,1
3,1447066.0,979.9,131.7,801.0,...,1
4,1447067.0,980.9,132.2,801.0,...,1


In [21]:
fix_in_problem_area = pd.concat([in_roi(fix_data[['axp', 'ayp']], ROIS['P1']),
                                 in_roi(fix_data[['axp', 'ayp']], ROIS['P2']),
                                 in_roi(fix_data[['axp', 'ayp']], ROIS['P3'])], axis=1).any(axis = 1)
first_fix_in_problem_area = np.where(fix_in_problem_area == True)[0][0]
fix_data_for_RF = fix_data[first_fix_in_problem_area:]

In [22]:
fix_in_matrix_area = pd.concat([in_roi(fix_data_for_RF[['axp', 'ayp']], ROIS['A']),
                                in_roi(fix_data_for_RF[['axp', 'ayp']], ROIS['B']),
                                in_roi(fix_data_for_RF[['axp', 'ayp']], ROIS['C']),
                                in_roi(fix_data_for_RF[['axp', 'ayp']], ROIS['D']),
                                in_roi(fix_data_for_RF[['axp', 'ayp']], ROIS['E']),
                                in_roi(fix_data_for_RF[['axp', 'ayp']], ROIS['F'])], axis = 1).any(axis = 1)
beh_data['time_of_first_fix_on_matrix_area'] = (fix_data_for_RF[fix_in_matrix_area].groupby('block').first()['stime'] - raw_data.groupby('block').first()['time']).fillna(0).reset_index()[0]

In [29]:
gb = beh_data.groupby('answers').sum() 
w = (gb['time_of_first_fix_on_matrix_area'] / (gb['rt'] * 1000.0))
w

answers
[1, 2, 2, 3, 3, 6]    0.053509
[1, 2, 3, 4, 5, 6]    0.008634
[1, 3, 3, 4, 4, 6]    0.045282
[1, 4, 4, 5, 5, 6]    0.117565
dtype: float64

# number of responses visited (RV) 
counted all response alternatives that were fixated at least once during the response time (following Bethell-Fox et al., 1984)

Modyfikacja! Teraz: liczba fiksacji + łączny czas trwania fiksacji na każdej z odpowiedzi. [time spent on each response alternative]

In [53]:
problems = yaml_data['list_of_blocks'][1]['experiment_elements'][1:]
problems += yaml_data['list_of_blocks'][2]['experiment_elements'][1:]
names = list()
for problem in problems:
    tmp = []
    for matrix in problem['matrix_info']:
        tmp.append(matrix['name'])
    names.append(tmp)
    
names = [x[3:] for x in names]
new_names = []
res = []
for x in names:
    new_names.append([int(w[-1]) for w in x])
for name, lab in zip(new_names, beh_data['answers'].map({
            '[1, 2, 3, 4, 5, 6]': "T", 
            '[1, 2, 2, 3, 3, 6]': 'H', 
            '[1, 3, 3, 4, 4, 6]': 'M',
            '[1, 4, 4, 5, 5, 6]': 'E'})):
    print name, lab,
#     one_pos = np.where(np.array(name) == 1)[0][0]
# #     two_pos = np.where(np.array(name) == 2)[0][0]
# #     three_pos = np.where(np.array(name) == 3)[0][0]
# #     four_pos = np.where(np.array(name) == 4)[0][0]
# #     five_pos = np.where(np.array(name) == 5)[0][0]
#     six_pos = np.where(np.array(name) == 6)[0][0]
#     name[one_pos] = '1_' + str(lab)
# #     name[two_pos] = '2_' + str(lab)
# #     name[three_pos] = '3_' + str(lab)
# #     name[four_pos] = '4_' + str(lab)
# #     name[five_pos] = '5_' + str(lab)
#     name[six_pos] = '6_' + str(lab)
    new_name = []
    for n in name:
        new_name.append(str(n) + '_' + str(lab))
    res.append(new_name)
    print name
    
    
new_names = res
# problems[0]['matrix_info']

[1, 5, 5, 6, 4, 4] T [1, 5, 5, 6, 4, 4]
[3, 4, 6, 4, 1, 3] T [3, 4, 6, 4, 1, 3]
[6, 3, 4, 1, 4, 3] T [6, 3, 4, 1, 4, 3]
[3, 1, 2, 3, 6, 2] E [3, 1, 2, 3, 6, 2]
[6, 1, 5, 4, 4, 5] M [6, 1, 5, 4, 4, 5]
[3, 6, 3, 1, 4, 4] M [3, 6, 3, 1, 4, 4]
[3, 3, 4, 4, 1, 6] H [3, 3, 4, 4, 1, 6]
[6, 3, 2, 2, 3, 1] E [6, 3, 2, 2, 3, 1]
[6, 1, 3, 3, 4, 4] M [6, 1, 3, 3, 4, 4]
[4, 4, 5, 1, 6, 5] M [4, 4, 5, 1, 6, 5]
[3, 1, 2, 3, 2, 6] H [3, 1, 2, 3, 2, 6]
[3, 1, 3, 6, 4, 4] M [3, 1, 3, 6, 4, 4]
[3, 1, 6, 2, 2, 3] E [3, 1, 6, 2, 2, 3]
[2, 3, 2, 3, 1, 6] H [2, 3, 2, 3, 1, 6]
[3, 1, 6, 3, 2, 2] M [3, 1, 6, 3, 2, 2]
[6, 2, 1, 3, 3, 2] H [6, 2, 1, 3, 3, 2]
[4, 5, 5, 6, 1, 4] H [4, 5, 5, 6, 1, 4]
[4, 5, 6, 5, 4, 1] H [4, 5, 6, 5, 4, 1]
[1, 5, 4, 6, 4, 5] H [1, 5, 4, 6, 4, 5]
[2, 6, 3, 3, 1, 2] E [2, 6, 3, 3, 1, 2]
[4, 3, 1, 4, 6, 3] E [4, 3, 1, 4, 6, 3]
[6, 4, 3, 4, 3, 1] E [6, 4, 3, 4, 3, 1]
[4, 6, 4, 1, 3, 3] H [4, 6, 4, 1, 3, 3]
[5, 6, 4, 5, 1, 4] M [5, 6, 4, 5, 1, 4]
[2, 3, 2, 3, 1, 6] M [2, 3, 2, 3, 1, 6]


In [54]:
fix_in_A = fix_data[in_roi(fix_data[['axp', 'ayp']], ROIS['A'])].groupby('block').sum()['dur']
fix_in_B = fix_data[in_roi(fix_data[['axp', 'ayp']], ROIS['B'])].groupby('block').sum()['dur']
fix_in_C = fix_data[in_roi(fix_data[['axp', 'ayp']], ROIS['C'])].groupby('block').sum()['dur']
fix_in_D = fix_data[in_roi(fix_data[['axp', 'ayp']], ROIS['D'])].groupby('block').sum()['dur']
fix_in_E = fix_data[in_roi(fix_data[['axp', 'ayp']], ROIS['E'])].groupby('block').sum()['dur']
fix_in_F = fix_data[in_roi(fix_data[['axp', 'ayp']], ROIS['F'])].groupby('block').sum()['dur']

In [55]:
missing_blocks = set(range(1, 46)) - set(fix_in_A.index)
for i in list(missing_blocks):
     fix_in_A.ix[i] = 0 
fix_in_A.sort_index(inplace=True)
fix_in_A = fix_in_A.reset_index()

missing_blocks = set(range(1, 46)) - set(fix_in_B.index)
for i in list(missing_blocks):
     fix_in_B.ix[i] = 0 
fix_in_B.sort_index(inplace=True)
fix_in_B = fix_in_B.reset_index()

missing_blocks = set(range(1, 46)) - set(fix_in_C.index)
for i in list(missing_blocks):
     fix_in_C.ix[i] = 0 
fix_in_C.sort_index(inplace=True)
fix_in_C = fix_in_C.reset_index()

missing_blocks = set(range(1, 46)) - set(fix_in_D.index)
for i in list(missing_blocks):
     fix_in_D.ix[i] = 0 
fix_in_D.sort_index(inplace=True)
fix_in_D = fix_in_D.reset_index()

missing_blocks = set(range(1, 46)) - set(fix_in_E.index)
for i in list(missing_blocks):
     fix_in_E.ix[i] = 0 
fix_in_E.sort_index(inplace=True)
fix_in_E = fix_in_E.reset_index()

missing_blocks = set(range(1, 46)) - set(fix_in_F.index)
for i in list(missing_blocks):
     fix_in_F.ix[i] = 0 
fix_in_F.sort_index(inplace=True)
fix_in_F = fix_in_F.reset_index()

# print fix_in_A.columns
fix_in_A.columns = ['block', 'dur_in_A']
fix_in_B.columns = ['blk', 'dur_in_B']
fix_in_C.columns = ['blk', 'dur_in_C']
fix_in_D.columns = ['blk', 'dur_in_D']
fix_in_E.columns = ['blk', 'dur_in_E']
fix_in_F.columns = ['blk', 'dur_in_F']


In [56]:
fix_dur = pd.concat([fix_in_A, fix_in_B, fix_in_C, fix_in_D,fix_in_E,fix_in_F], axis = 1).drop('blk', 1)
# Training removal
fix_dur = fix_dur[fix_dur.block != 1]
fix_dur = fix_dur[fix_dur.block != 2]
fix_dur = fix_dur[fix_dur.block != 3]
fix_dur.reset_index(inplace=True)
fix_dur.head()

,index,block,dur_in_A,dur_in_B,dur_in_C,dur_in_D,dur_in_E,dur_in_F
0,3,4,4097,1044,1599,1899,779,1943.0
1,4,5,3887,1323,273,3085,5228,459.0
2,5,6,808,2314,274,4274,792,144.0
3,6,7,1474,5541,1039,5270,545,829.0
4,7,8,1273,2229,684,3479,1246,1533.0


In [65]:
d = collections.defaultdict(list)
for l, row in zip(new_names, fix_dur.iterrows()):
    for k, v in  zip(l, row[1][2:]):
        d[k].append(v)
d.pop('1_T', None)
d.pop('6_T', None)

for k,v in d.iteritems():
    print k, [val for val in v if not math.isnan(val)]

4_H [4.0, 20.0, 14.0, 0.0, 4.0, 3.0, 1.0, 1.0, 0.0, 0.0, 25.0, 1.0, 13.0, 12.0, 15.0, 14.0, 2.0, 1.0, 8.0, 5.0]
3_T [21.0, 4.0, 13.0, 1.0]
5_E [3.0, 6.0, 35.0, 7.0, 12.0, 9.0]
4_M [19.0, 8.0, 18.0, 5.0, 7.0, 4.0, 10.0, 8.0, 5.0, 1.0, 5.0, 1.0, 4.0, 1.0, 2.0, 6.0, 14.0, 2.0]
5_H [4.0, 4.0, 4.0, 4.0, 1.0, 4.0, 20.0, 24.0, 5.0, 1.0]
5_M [3.0, 10.0, 3.0, 15.0, 5.0, 21.0, 7.0, 2.0, 5.0, 2.0]
4_E [21.0, 16.0, 0.0, 0.0, 10.0, 1.0, 38.0, 10.0, 15.0, 28.0, 18.0, 2.0]
2_H [12.0, 14.0, 9.0, 1.0, 25.0, 9.0, 6.0, 15.0]
3_E [9.0, 27.0, 15.0, 41.0, 5.0, 28.0, 22.0, 3.0, 20.0, 10.0, 0.0, 0.0, 15.0, 7.0, 13.0, 6.0, 4.0, 48.0]
2_M [19.0, 11.0, 3.0, 2.0, 12.0, 31.0, 11.0, 18.0]
5_T [6.0, 8.0]
3_H [34.0, 10.0, 17.0, 27.0, 14.0, 1.0, 10.0, 10.0, 10.0, 7.0, 13.0, 7.0, 4.0, 1.0, 2.0, 3.0, 9.0]
3_M [21.0, 9.0, 8.0, 16.0, 4.0, 2.0, 33.0, 12.0, 7.0, 6.0, 18.0, 4.0, 3.0, 25.0, 31.0, 5.0]
2_E [6.0, 4.0, 7.0, 28.0, 3.0, 31.0, 5.0, 8.0, 11.0, 2.0, 7.0, 4.0]
4_T [4.0, 10.0, 9.0, 19.0, 2.0, 4.0]
1_E [29.0, 29.0, 43.0

In [58]:
fix_in_A = fix_data[in_roi(fix_data[['axp', 'ayp']], ROIS['A'])].groupby('block').count()['dur']
fix_in_B = fix_data[in_roi(fix_data[['axp', 'ayp']], ROIS['B'])].groupby('block').count()['dur']
fix_in_C = fix_data[in_roi(fix_data[['axp', 'ayp']], ROIS['C'])].groupby('block').count()['dur']
fix_in_D = fix_data[in_roi(fix_data[['axp', 'ayp']], ROIS['D'])].groupby('block').count()['dur']
fix_in_E = fix_data[in_roi(fix_data[['axp', 'ayp']], ROIS['E'])].groupby('block').count()['dur']
fix_in_F = fix_data[in_roi(fix_data[['axp', 'ayp']], ROIS['F'])].groupby('block').count()['dur']

In [59]:
missing_blocks = set(range(1, 39)) - set(fix_in_A.index)
for i in list(missing_blocks):
     fix_in_A.ix[i] = 0 
fix_in_A.sort_index(inplace=True)
fix_in_A = fix_in_A.reset_index()

missing_blocks = set(range(1, 39)) - set(fix_in_B.index)
for i in list(missing_blocks):
     fix_in_B.ix[i] = 0 
fix_in_B.sort_index(inplace=True)
fix_in_B = fix_in_B.reset_index()

missing_blocks = set(range(1, 39)) - set(fix_in_C.index)
for i in list(missing_blocks):
     fix_in_C.ix[i] = 0 
fix_in_C.sort_index(inplace=True)
fix_in_C = fix_in_C.reset_index()

missing_blocks = set(range(1, 39)) - set(fix_in_D.index)
for i in list(missing_blocks):
     fix_in_D.ix[i] = 0 
fix_in_D.sort_index(inplace=True)
fix_in_D = fix_in_D.reset_index()

missing_blocks = set(range(1, 39)) - set(fix_in_E.index)
for i in list(missing_blocks):
     fix_in_E.ix[i] = 0 
fix_in_E.sort_index(inplace=True)
fix_in_E = fix_in_E.reset_index()

missing_blocks = set(range(1, 39)) - set(fix_in_F.index)
for i in list(missing_blocks):
     fix_in_F.ix[i] = 0 
fix_in_F.sort_index(inplace=True)
fix_in_F = fix_in_F.reset_index()

# print fix_in_A.columns
fix_in_A.columns = ['block', 'no_in_A']
fix_in_B.columns = ['blk', 'no_in_B']
fix_in_C.columns = ['blk', 'no_in_C']
fix_in_D.columns = ['blk', 'no_in_D']
fix_in_E.columns = ['blk', 'no_in_E']
fix_in_F.columns = ['blk', 'no_in_F']


In [60]:
fix_dur = pd.concat([fix_in_A, fix_in_B, fix_in_C, fix_in_D,fix_in_E,fix_in_F], axis = 1).drop('blk', 1)
fix_dur = fix_dur[fix_dur.block != 1]
fix_dur = fix_dur[fix_dur.block != 2]
fix_dur = fix_dur[fix_dur.block != 3]
fix_dur.reset_index(inplace=True)

In [61]:
fix_dur.head()

,index,block,no_in_A,no_in_B,no_in_C,no_in_D,no_in_E,no_in_F
0,3,4,21,6.0,8,12,4.0,10.0
1,4,5,21,9.0,1,19,28.0,4.0
2,5,6,5,13.0,2,19,4.0,1.0
3,6,7,9,29.0,6,27,3.0,4.0
4,7,8,7,11.0,3,19,8.0,10.0


In [62]:
d = collections.defaultdict(list)
for l, row in zip(new_names, fix_dur.iterrows()):
    for k, v in  zip(l, row[1][2:]):
        d[k].append(v)

d.pop('1_T', None)
d.pop('6_T', None)
for k,v in d.iteritems():
    print k, np.sum([val for val in v if not math.isnan(val)])

4_H 143.0
3_T 39.0
5_E 72.0
4_M 120.0
5_H 71.0
5_M 73.0
4_E 159.0
2_H 91.0
3_E 273.0
2_M 107.0
5_T 14.0
3_H 179.0
3_M 204.0
2_E 116.0
4_T 48.0
1_E 209.0
1_M 187.0
1_H 180.0
6_E 130.0
6_M 135.0
6_H 88.0


# Pupil size

In [63]:
fix_data.head()

,stime,etime,dur,axp,ayp,aps,eye,block
0,1447070,1447093,24,980.6,133.9,801,R,1
1,1447172,1447294,123,1082.1,371.4,825,R,1
2,1447379,1447510,132,82.8,229.8,867,R,1
3,1447562,1447878,317,35.1,20.1,871,R,1
4,1447940,1448077,138,568.7,264.4,910,R,1


In [64]:
fix_in_matrix_area = pd.concat([in_roi(fix_data[['axp', 'ayp']], ROIS['A']),
                                in_roi(fix_data[['axp', 'ayp']], ROIS['B']),
                                in_roi(fix_data[['axp', 'ayp']], ROIS['C']),
                                in_roi(fix_data[['axp', 'ayp']], ROIS['D']),
                                in_roi(fix_data[['axp', 'ayp']], ROIS['E']),
                                in_roi(fix_data[['axp', 'ayp']], ROIS['F'])], axis = 1).any(axis = 1)
avg_pupul_size = fix_data[fix_in_matrix_area].groupby('block').mean()['aps']

missing_blocks = set(range(1, 39)) - set(avg_pupul_size.index)
for i in list(missing_blocks):
     avg_pupul_size.ix[i] = 0 
avg_pupul_size.sort_index(inplace=True)
avg_pupul_size = avg_pupul_size.reset_index()

beh_data['avg_pupil_size'] = avg_pupul_size['aps']
w = beh_data.groupby('answers').mean()['avg_pupil_size']
w

answers
[1, 2, 2, 3, 3, 6]     956.498449
[1, 2, 3, 4, 5, 6]    1023.741626
[1, 3, 3, 4, 4, 6]     905.367596
[1, 4, 4, 5, 5, 6]     972.742194
Name: avg_pupil_size, dtype: float64